In [1]:
import numpy as np
import pandas as pd

In [2]:
df = pd.read_csv("C:\\Users\\Anil\\Downloads\\training_twitter.csv",delimiter=',')
x_train=df["text"].values
y_train=df["airline_sentiment"].values

In [3]:
from nltk.tokenize import word_tokenize

In [4]:
documents=[]
for j in range(df.shape[0]):        
    documents.append((word_tokenize(x_train[j]),y_train[j]))

In [5]:
from nltk.stem import WordNetLemmatizer

In [6]:
lemmatizer=WordNetLemmatizer()

In [7]:
from nltk.corpus import wordnet
def get_simple_pos(tag):
    
    if tag.startswith('J'):
        return wordnet.ADJ
    elif tag.startswith('V'):
        return wordnet.VERB
    elif tag.startswith('N'):
        return wordnet.NOUN
    elif tag.startswith('R'):
        return wordnet.ADV
    else:
        return wordnet.NOUN

In [8]:
from nltk.corpus import stopwords
import string
stops=set(stopwords.words('english'))
punctuations=list(string.punctuation)
stops.update(punctuations)

In [9]:
stops.remove("no")
stops.remove("not")
stops.remove("aren't")
stops.remove("isn't")
stops.remove("couldn't")
stops.remove("didn't")
stops.remove("don't")
stops.remove("hadn't")
stops.remove("hasn't")
stops.remove("mightn't")
stops.remove("mustn't")
stops.remove("shouldn't")
stops.remove("weren't")
stops.remove("wouldn't")
stops.remove("won't")
stops.remove("aren")
stops.remove("isn")
stops.remove("couldn")
stops.remove("didn")
stops.remove("don")
stops.remove("hadn")
stops.remove("hasn")
stops.remove("mightn")
stops.remove("mustn")
stops.remove("shouldn")
stops.remove("weren")
stops.remove("wouldn")
stops.remove("won")

In [10]:
from nltk import pos_tag
def clean_review(words):
    output_words=[]
    for w in words:
        if w.lower() not in stops:
            pos=pos_tag([w])
            clean_word=lemmatizer.lemmatize(w,pos=get_simple_pos(pos[0][1]))
            output_words.append(clean_word.lower())
            
    return output_words

In [11]:
documents=[(clean_review(document),category)for document,category in documents]

In [12]:
x_train=[" ".join(document) for document,category in documents]

In [13]:
y_train=[category for document,category in documents]

In [14]:
#decide features
from sklearn.feature_extraction.text import CountVectorizer

In [15]:
count_vec=CountVectorizer(ngram_range=(1,3), min_df=0.030)
x_train_features=count_vec.fit_transform(x_train)
x_train_features

<10980x49 sparse matrix of type '<class 'numpy.int64'>'
	with 38023 stored elements in Compressed Sparse Row format>

In [16]:
features=count_vec.get_feature_names()

In [17]:
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
from sklearn.neighbors import KNeighborsClassifier

In [18]:
clf=SVC()
grid={'C':[1e2,1e3,1e4],'gamma':[1e-3,1e-4,5e-3]}

abc=GridSearchCV(clf,grid)
abc.fit(x_train_features,y_train)

C:\Users\Anil\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:2053: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)


GridSearchCV(cv='warn', error_score='raise-deprecating',
       estimator=SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto_deprecated',
  kernel='rbf', max_iter=-1, probability=False, random_state=None,
  shrinking=True, tol=0.001, verbose=False),
       fit_params=None, iid='warn', n_jobs=None,
       param_grid={'C': [100.0, 1000.0, 10000.0], 'gamma': [0.001, 0.0001, 0.005]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

In [19]:
clf=SVC(C=100,gamma=0.08)
clf.fit(x_train_features,y_train)

SVC(C=100, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma=0.08, kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

In [20]:
from sklearn.ensemble import RandomForestClassifier

In [21]:
rfc=RandomForestClassifier()
rfc.fit(x_train_features,y_train)

C:\Users\Anil\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=None,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [22]:
df1 = pd.read_csv("C:\\Users\\Anil\\Downloads\\test_twitter.csv",delimiter=',')
#x_train=df["text"].values
x_test=df1["text"]

In [23]:
documents_test=[]
for j in range(df1.shape[0]):        
    documents_test.append(word_tokenize(x_test[j]))

In [24]:
documents_test=[(clean_review(document)) for document in documents_test]

In [25]:
x_test=[" ".join(document) for document in documents_test]

In [26]:
x_test_features=count_vec.transform(x_test)

In [27]:
y_test=clf.predict(x_test_features)

In [28]:
y_test=abc.predict(x_test_features)

In [29]:
y_test=rfc.predict(x_test_features)

In [30]:
(y_test=='negative').sum()

2545